In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
from seaborn import color_palette
import numpy as np
from util import load_results_from_dir

sns.set_palette('Set1')
plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "serif",
        # "font.serif": ["Computer Modern Roman", "Times New Roman"] + plt.rcParams["font.serif"],
        "text.latex.preamble": r"\usepackage[T1]{fontenc} \usepackage[utf8]{inputenc} \usepackage{lmodern}",
    }
)
figsize=(5,2.2)
dpi = 150
parameters = ['scheduler', 'N_w', 'N_d', 'N_s', 'W_hat', 'pi', 'sigma', 'epsilon', 'int_feas_tol', 'v', 'r_charge', 'r_deplete', 'B_min', 'B_max', 'B_start']

# Load data

In [ ]:
rootdir = "../out/villalvernia.elrond/optimal_perf"

In [ ]:
df = load_results_from_dir(rootdir)
df['experiment_type'] = None
df.loc[df.scheduler == 'naivescheduler', 'experiment_type'] = 'naive'
df.loc[(df.scheduler == 'milpscheduler') & (df.pi == np.inf) & (df.sigma == 1), 'experiment_type'] = 'optimal'
df.loc[(df.scheduler == 'milpscheduler') & (df.pi == np.inf) & (df.sigma == 2), 'experiment_type'] = 'sigma2'
df.loc[(df.scheduler == 'milpscheduler') & (df.pi == np.inf) & (df.sigma == 3), 'experiment_type'] = 'sigma3'
df.loc[(df.scheduler == 'milpscheduler') & (df.W_hat == 10) & (df.pi == 8) & (df.sigma == 1), 'experiment_type'] = 'fixed_w_hat_10'
df.loc[(df.scheduler == 'milpscheduler') & (df.W_hat == 15) & (df.pi == 13) & (df.sigma == 1), 'experiment_type'] = 'fixed_w_hat_15'

# filter out non-relevant measurements
df = df[lambda x: x.trial == 1]
df = df[lambda x: x.voxel_size <= 7]
df = df[lambda x: x.B_min == 0.8]

# remove duplicates
idx = df[['N_s', 'N_d', 'voxel_size', 'W_hat', 'pi', 'sigma', 'epsilon', 'int_feas_tol', 'trial']].drop_duplicates().index
df = df.loc[idx]

In [ ]:
print(f"Number of samples: {df.shape[0]:,}")
display(df.experiment_type.value_counts().reset_index())

# $\sigma$ changes

In [ ]:
# experiment_types = ['optimal', 'sigma2', 'sigma3', 'fixed_w_hat']
experiment_types = ['optimal', 'sigma2', 'sigma3', 'naive']
# experiment_types = ['optimal', 'sigma2', 'sigma3']
df_no_rescheduling = df[lambda x: x.experiment_type.isin(experiment_types)]
# df_no_rescheduling = df_no_rescheduling[lambda x: x.trial == 1]  # remove if more trials necessary
print(f"Number of samples: {df_no_rescheduling.shape[0]:,}")
display(df_no_rescheduling.experiment_type.value_counts().reset_index())

In [ ]:
label_translation = {
    'naive': 'Naive',
    'optimal': 'Optimal', 
    'sigma2': r'$\sigma = 2$',
    'sigma3': r'$\sigma = 3$',
}

## Execution time

In [ ]:
plt.figure(figsize=figsize, dpi=dpi)

hue_order = ['optimal', 'sigma2', 'sigma3', 'naive']
ax = sns.lineplot(data=df_no_rescheduling, x='voxel_size', y='execution_time', hue='experiment_type', hue_order=hue_order, marker='h')
plt.xlabel("$N_w$")
plt.ylabel("Execution time (s)")

handles, labels = ax.get_legend_handles_labels()
new_labels = [label_translation[i] for i in labels]
plt.legend(handles, new_labels, bbox_to_anchor=(1.05, .5), borderaxespad=0, loc='center left')

plt.savefig("../out/figures/optimal_computation_time/sigma.execution_time.line.pdf", bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=figsize, dpi=dpi)

hue_order = ['optimal', 'sigma2', 'sigma3', 'naive']
sns.barplot(data=df_no_rescheduling, x='N_w', y='execution_time', hue='experiment_type', hue_order=hue_order, edgecolor='black', alpha=0.7)
plt.xlabel("$N_w$")
plt.ylabel("Execution time (s)")

handles, labels = ax.get_legend_handles_labels()
new_labels = [label_translation[i] for i in labels]
plt.legend(handles, new_labels, bbox_to_anchor=(1.05, .5), borderaxespad=0, loc='center left')
plt.grid(axis='y')

plt.savefig("../out/figures/optimal_computation_time/sigma.execution_time.bar.pdf", bbox_inches='tight')
plt.show()

## Solve time

In [ ]:
plt.figure(figsize=figsize, dpi=dpi)

hue_order = ['optimal', 'sigma2', 'sigma3']
ax = sns.lineplot(data=df_no_rescheduling, x='N_w', y='t_solve_total', hue='experiment_type', hue_order=hue_order, marker='h')
plt.xlabel("$N_w$")
plt.ylabel("$t_{solve}$ [log]")
plt.yscale('log')

handles, labels = ax.get_legend_handles_labels()
new_labels = [label_translation[i] for i in labels]
plt.legend(handles, new_labels, bbox_to_anchor=(1.05, .5), borderaxespad=0, loc='center left')

plt.savefig("../out/figures/optimal_computation_time/sigma.t_solve_total.line.pdf", bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=figsize, dpi=dpi)

hue_order = ['optimal', 'sigma2', 'sigma3']
sns.barplot(data=df_no_rescheduling, x='N_w', y='t_solve_total', hue='experiment_type', hue_order=hue_order, edgecolor='black', alpha=0.7)
plt.xlabel("$N_w$")
plt.ylabel("Scheduling time (s)")
plt.yscale('log')

handles, labels = ax.get_legend_handles_labels()
new_labels = [label_translation[i] for i in labels]
plt.legend(handles, new_labels, bbox_to_anchor=(1.05, .5), borderaxespad=0, loc='center left')
plt.grid(axis='y')

plt.savefig("../out/figures/optimal_computation_time/sigma.t_solve_total.bar.pdf", bbox_inches='tight')
plt.show()

# $\hat{W}$ changes

In [ ]:
label_translation = {
    'optimal': 'Optimal $(\Sigma)$', 
    'fixed_w_hat_10': r'$\hat{W} = 10$ $(\Sigma)$',
    'fixed_w_hat_15': r'$\hat{W} = 15$ $(\Sigma)$',
    'fixed_w_hat_10_mean': r'$\hat{W} = 10$ $(\mu)$',
    'fixed_w_hat_15_mean': r'$\hat{W} = 15$ $(\mu)$',
}# Plot results of fixed W_hat

In [ ]:
experiment_types = ['optimal', 'fixed_w_hat_10', 'fixed_w_hat_15']
df_fixed_W_hat = df[lambda x: x.experiment_type.isin(experiment_types)]
# df_fixed_W_hat = df_fixed_W_hat[lambda x: x.trial == 1]  # remove if more trials necessary

## Execution time 

In [ ]:
label_translation = {
    'optimal': 'Optimal', 
    'fixed_w_hat_10': r'$\hat{N}_w = 10$',
    'fixed_w_hat_15': r'$\hat{N}_w = 15$',
}

In [ ]:
_, ax = plt.subplots(figsize=figsize, dpi=dpi)

hue_order = ['optimal', 'fixed_w_hat_15', 'fixed_w_hat_10']
sns.barplot(data=df_fixed_W_hat, x='N_w', y='execution_time', hue='experiment_type', hue_order=hue_order, edgecolor='black', alpha=0.7, ax=ax)

plt.xlabel("$N_w$")
plt.ylabel("Execution time (s)")

handles, labels = ax.get_legend_handles_labels()
new_labels = [label_translation[i] for i in labels]
plt.legend(handles, new_labels, bbox_to_anchor=(1.05, .5), borderaxespad=0, loc='center left')

plt.savefig("../out/figures/optimal_computation_time/N_w.execution_time.bar.pdf", bbox_inches='tight')
plt.show()

## Solve times

In [ ]:
label_translation = {
    'optimal_total': 'Optimal', 
    'fixed_w_hat_10_total': r'$\hat{N}_w = 10$ $(\Sigma)$',
    'fixed_w_hat_15_total': r'$\hat{N}_w = 15$ $(\Sigma)$',
    'fixed_w_hat_10_mean': r'$\hat{N}_w = 10$ $(\mu)$',
    'fixed_w_hat_15_mean': r'$\hat{N}_w = 15$ $(\mu)$',
}
hue_order = ['optimal_total', 'fixed_w_hat_15_total', 'fixed_w_hat_10_total', 'fixed_w_hat_15_mean', 'fixed_w_hat_10_mean']

In [ ]:
df_plot_total = df_fixed_W_hat.rename(columns={'t_solve_total': 'y'})
df_plot_total['experiment_type'] = df_plot_total['experiment_type'].astype(str) + "_total"
df_plot_mean = df_fixed_W_hat[lambda x: x.experiment_type != 'optimal'].rename(columns={'t_solve_mean': 'y'})
df_plot_mean['experiment_type'] = df_plot_mean['experiment_type'].astype(str) + "_mean"
df_plot_both = pd.concat([df_plot_total, df_plot_mean])
print(f"Number of samples (sum):  {df_plot_total.shape[0]:,}")
print(f"Number of samples (mean): {df_plot_mean.shape[0]:,}")

In [ ]:
_, ax = plt.subplots(figsize=figsize, dpi=dpi)

cp = color_palette()
palette = [cp[0], cp[1], cp[2], cp[1], cp[2]]
sns.barplot(data=df_plot_both, x='N_w', y='y', hue='experiment_type', hue_order=hue_order, edgecolor='black', alpha=0.7, palette=palette, ax=ax)
           
plt.xlabel("$N_w$")
plt.ylabel("Scheduling time (s)")
plt.yscale('log')

handles, labels = ax.get_legend_handles_labels()
for i in [3, 4]:
    handle = handles[i]
    for rect in handle:
        rect.set_linestyle("--")
new_labels = [label_translation[i] for i in labels]
plt.legend(handles, new_labels, bbox_to_anchor=(1.05, .5), borderaxespad=0, loc='center left')
plt.grid(axis='y')

plt.savefig("../out/figures/optimal_computation_time/N_w.t_solve.bar.pdf", bbox_inches='tight')
plt.show()

# Combine experiments

## Execution time

In [ ]:
df_plot = df[lambda x: x.experiment_type.isin(['optimal', 'fixed_w_hat_15', 'sigma2', 'naive'])]

In [ ]:
label_translation = {
    'optimal': 'Optimal',
    'fixed_w_hat_15': '$\hat{N}_w = 15$',
    'sigma2': '$\sigma = 2$',
    'naive': 'Naive',
}

In [ ]:
_, ax = plt.subplots(figsize=figsize, dpi=dpi)

hue_order = ['optimal', 'fixed_w_hat_15', 'sigma2', 'naive']
sns.barplot(data=df_plot, x='N_w', y='execution_time', hue='experiment_type', hue_order=hue_order, edgecolor='black', alpha=0.7, ax=ax, ci=None)

plt.xlabel("$N_w$")
plt.ylabel("Execution time (s)")

handles, labels = ax.get_legend_handles_labels()
new_labels = [label_translation[i] for i in labels]
# plt.legend(handles, new_labels, bbox_to_anchor=(1.05, .5), borderaxespad=0, loc='center left')
plt.legend(handles, new_labels, frameon=True, edgecolor='white', framealpha=.9, fontsize=8)
plt.grid(axis='y')

plt.savefig("../out/figures/optimal_computation_time/combined.execution_time.bar.pdf", bbox_inches='tight')
plt.show()

## Solve times

In [ ]:
label_translation = {
    'optimal_total': 'Optimal',
    'fixed_w_hat_15_total': '$\hat{N}_w = 15$ $(\Sigma)$',
    'fixed_w_hat_15_mean': '$\hat{N}_w = 15$ $(\mu)$',
    'sigma2_total': '$\sigma = 2$',
    'naive_total': 'Naive',
}

In [ ]:
df_plot_total = df[lambda x: x.experiment_type.isin(['optimal', 'fixed_w_hat_15', 'sigma2', 'naive'])].copy()
df_plot_mean = df[lambda x: x.experiment_type == 'fixed_w_hat_15'].copy()

df_plot_total['y'] = df_plot_total.t_solve_total
df_plot_total['experiment_type'] = df_plot_total['experiment_type'].astype(str) + "_total"
df_plot_mean['y'] = df_plot_mean.t_solve_mean
df_plot_mean['experiment_type'] = df_plot_mean['experiment_type'].astype(str) + "_mean"

df_plot = pd.concat([df_plot_total, df_plot_mean])

In [ ]:
_, ax = plt.subplots(figsize=figsize, dpi=dpi)

cp = color_palette()
palette = [cp[0], cp[1], cp[1], cp[2], cp[3]]

hue_order = ['optimal_total', 'fixed_w_hat_15_total', 'fixed_w_hat_15_mean', 'sigma2_total']
sns.barplot(data=df_plot, x='N_w', y='y', hue='experiment_type', hue_order=hue_order, edgecolor='black', alpha=0.7, ax=ax, palette=palette, ci="sd")
ax.axhline(y=600, color='r', linestyle=":", zorder=-1)

plt.xlabel("$N_w$")
plt.ylabel("Scheduling time (s)")

handles, labels = ax.get_legend_handles_labels()
for rec in handles[2]:
    rec.set_linestyle('--')
    rec.set_hatch("/" * 9)
    rec.set_fill(None)
    rec.set_linewidth(0)
    rec.set_color(cp[1])    


new_labels = [label_translation[i] for i in labels]

# plt.legend(handles, new_labels, bbox_to_anchor=(1.05, .5), borderaxespad=0, loc='center left')
plt.legend(handles, new_labels, frameon=True, edgecolor='white', framealpha=.9, fontsize=8)
plt.grid(axis='y')
plt.yscale('log')

plt.savefig("../out/figures/optimal_computation_time/combined.t_solve.bar.pdf", bbox_inches='tight')
plt.show()